# CSCN8020 Q Learning Taxi


The environment we define will contain:
- 500 discrete states
- 6 discrete actions
- Reward structure:
    - -1 per step
    - +20 for correct passenger drop-off
    - -10 for illegal pickup or drop-off

The goal is to learn the optimal policy that maximizes cumulative reward by efficiently transporting passengers.

Legend:
- Agent: Taxi
- Enviornment: Taxi-v3 grid world
- State Space: 500 discrete states
- Action Space: 6 discrete actions
- Policy: ε-greedy
- Value function: Q(s,a)
- Learning Type: Model free, off-policy, Temporal difference

The Q-Learning update rule:
Q(s,a) ← Q(s,a) + α [r + γ max_a' Q(s',a') − Q(s,a)]

Where:
- α = learning rate
- γ  = discount factor
- ε =  exploration factor ( ε-greedy policy)



## 1. Import & Setup

In [ ]:
from idlelib.rpc import request_queue

import gymnasium as gym
import numpy as np
import random
import logging
import matplotlib.pyplot as plt
import time

# Setup logging
logging.basicConfig(
    filename='taxi.log',
    filemode='w',
    level=logging.INFO,
    format="%(message)s"
)

logger = logging.getLogger()

Imports explanation:
- gymnasium: provides Taxi environment for experiment
- numpy: used for Q-table and math operations
- random: used for epsilon-greedy exploration
- logging: used to store metrics into a file that can be used to analyze and compare info
- matplotlib: used for plotting performance curves

The Q-table will be a 500 x 6 matrix
Each row corresponds to a state while each column corresponds to an action.

## 2. Initialize Environment & Analysis

Observation space: 500 states

State encoding:
((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination

Breakdown:
- 25 taxi positions
- 5 passenger locations
- 4 destination locations

Action Space (6 actions):
0: South (Down)
1: North (Up)
2: East (Right)
3: West (Left)
4: Pickup passenger
5: Drop off passenger

Reward Structure:
- -1per step
- +20 successful passenger drop-off
- -10 illegal pickup or drop off

The step penalty encourages shortest-path behavior
The +20 reward propagates backward via bootstrapping which is to say, the agent updates its current state-value estimate based on the estimated value of the previous state. Over multiple episodes the +20 reward is passed backward a step at a time which eventually points directly toward the goal.

In [ ]:
env = gym.make("Taxi-v3")

state_size = env.observation_space.n
action_size = env.action_space.n #.n indicates that the environment is using a discrete space and the .n stands for the number and it returns the absolute count of discrete elements within that specific space

print("State Size: ", state_size)
print("Action Size: ", action_size)

## 3. Q-Learning Implementation

We implement tabular Q-Learning:
This means the agent uses a 2D matrix(states as rows, actions as columns) to store expected future rewards, continuously updating these values through experience to learn the best possible action for any given situation

Q-table dimensions
500 x 6

Policy:
ε-greedy

Update:
Q(s,a) += α [ r + γ max Q(s',a') − Q(s,a) ]

The TD error is
δ = r + γ max Q(s',a') − Q(s,a)

If δ > 0 → underestimation
If δ < 0 → overestimation

The algorithm is model-free and off-policy.


In [ ]:
def train_q_learning(alpha, gamma, epsilon, episodes=5000):

    Q  = np.zeros([state_size, action_size])

    steps_per_episode = []
    rewards_per_episode = []

    for episode in range(episodes):

        state, _ = env.reset()
        done = False
        total_reward = 0
        steps = 0

        while not done:
            #ε-greedy action selection
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            #Q-Update
            Q[state, action] += alpha * (
                reward + gamma * np.max(Q[next_state]) - Q[state, action]
            ) # This line of code adjusts the current q-value byt taking a small step (determined by the learning rate alpha) toward the newly observed reality, which is the immediate reward plus the best possible estimated future value

            state = next_state
            total_reward += reward
            steps += 1

        steps_per_episode.append(steps)
        rewards_per_episode.append(total_reward)

        logger.info(f"{alpha},{gamma},{epsilon},{episode},{steps},{total_reward}")

    return Q, steps_per_episode, rewards_per_episode



## 4. Metric Reporting

The following metrics will be reported:
1. Total Episodes
2. Total steps per episode
3. Average return per episode
Each is computed and loged for each experiment

In [ ]:
def summarize_results(name, steps, rewards):

    total_episodes = len(rewards)
    avg_return = np.mean(rewards)
    avg_steps = np.mean(steps)

    print(f"\n===== {name} =====")
    print("Total Episodes:", total_episodes)
    print("Average Return:", avg_return)
    print("Average Steps:", avg_steps)

    logger.info(f"\n===== {name} =====")
    logger.info(f"Total Episodes: {total_episodes}")
    logger.info(f"Average Return: {avg_return}")
    logger.info(f"Average Steps: {avg_steps}")

## 5. Base Training run

In [ ]:
alpha = 0.1 #Learning Rate
gamma = 0.9 # Discount Factor
epsilon = 0.1 #Exploration Rate

Q_base, steps_base, rewards_base, = train_q_learning(alpha, gamma, epsilon)

summarize_results("Base_Run", steps_base, rewards_base)

## 6. Plot Base Metrics

In [ ]:
plt.figure()
plt.plot(rewards_base)
plt.title("Base Run: Reward per Episode")
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.show()

plt.figure()
plt.plot(steps_base)
plt.title("Base Run: Steps per Episode")
plt.xlabel("Episode")
plt.ylabel("Steps")
plt.show()